# 1. Dependencies

In [2]:
import cv2
import re
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp  
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

# Make sure in a python 3.8 env
# pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

# 2. Functions

In [3]:
mp_holistic = mp.solutions.holistic # bringing the holstic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction - detecting using mediapipe
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections
    
# CAN CHANGE THE COLOURS OF THESE TO MAKE IT DIFFERENT 
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# Extracting data points
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 2. Data Gathering

In [4]:
INPUT_VIDEO_PATH = r'C:\Users\Tommaso\Google Drive\Current Courses\COMP9444\CodingTasks\Dataset'

# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Videos are going to be 30 frames in length
SEQUENCE_LENGTH = 30

label_map = {}
no_sequences = {}
actions = []
# VIDEO NAMING CONVENTION = "ACTION.SEQUENCE.mp4" 
# e.g. "V.001.mp4"
for i,vid in enumerate(os.listdir(INPUT_VIDEO_PATH)):
    print(vid,i)
    string = vid.split('.')
    action = string[0]
    sequence = int(string[1])
    cap = cv2.VideoCapture(vid)
    label_map[action] = i
    
    if action in no_sequences:
        no_sequences[action] += 1
    else:
       no_sequences[action] = 1
      
    if action not in actions:
        actions.append(action)
    try: 
        os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
    except:
        print('Directory Already Exists, passing file')
        continue
        
    vid_loc = os.path.join(INPUT_VIDEO_PATH,vid)
    cap = cv2.VideoCapture(vid_loc)
    # Set mediapipe model 
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # this is code for reading from a video file
        frame_num = 0
        while(cap.isOpened() and frame_num < SEQUENCE_LENGTH):
            # Read feed
            ret, frame = cap.read()
            
            if ret:
                image, results = mediapipe_detection(frame, holistic)
                # Draw landmarks
                draw_styled_landmarks(image, results)
                cv2.waitKey(1)
                
                # UNCOMMENT THIS TO SEE THE VIDEO DISPLAYED
                # cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                #                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                # # # Show to screen
                # cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                frame_num += 1
            else:
                break
            
    cap.release()
    cv2.destroyAllWindows()



B.000.mp4 0
Directory Already Exists, passing file
B.001.mp4 1
Directory Already Exists, passing file
B.002.mp4 2
Directory Already Exists, passing file
B.003.mp4 3
Directory Already Exists, passing file
B.004.mp4 4
Directory Already Exists, passing file
B.005.mp4 5
Directory Already Exists, passing file
B.006.mp4 6
Directory Already Exists, passing file
B.007.mp4 7
Directory Already Exists, passing file
B.008.mp4 8
Directory Already Exists, passing file
B.009.mp4 9
Directory Already Exists, passing file
B.010.mp4 10
Directory Already Exists, passing file
B.011.mp4 11
Directory Already Exists, passing file
B.012.mp4 12
Directory Already Exists, passing file
B.013.mp4 13
Directory Already Exists, passing file
B.014.mp4 14
Directory Already Exists, passing file
B.015.mp4 15
Directory Already Exists, passing file
B.016.mp4 16
Directory Already Exists, passing file
B.017.mp4 17
Directory Already Exists, passing file
B.018.mp4 18
Directory Already Exists, passing file
B.019.mp4 19
Directory

# 3. Data Preprocessing

In [5]:
print(no_sequences)
print(actions)
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []
for action in actions:
    for sequence in range(20):
        window = []
        for frame_num in range(SEQUENCE_LENGTH):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence+1), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

print(label_map)
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

{'B': 24, 'C': 26, 'F': 29}
['B', 'C', 'F']
{'B': 0, 'C': 1, 'F': 2}


# 4. Build and Train LSTM Neural Network

In [6]:

# Reasons for doing this
# - less data to produce a hyper accurate model
# - much denser neural network (rather than 30 40 million paramters have BLANK)
# - It was a whole heap faster in detecting in real time
print(X.shape)
actions = np.asarray(actions)
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))
#model.summary()
#model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])



(60, 30, 1662)
Epoch 1/2000
2/2 [==============================] - 3s 479ms/step - loss: 1.1116 - categorical_accuracy: 0.3889
Epoch 2/2000
2/2 [==============================] - 0s 73ms/step - loss: 12.3404 - categorical_accuracy: 0.3330
Epoch 3/2000
2/2 [==============================] - 0s 84ms/step - loss: 7.4743 - categorical_accuracy: 0.3056
Epoch 4/2000
2/2 [==============================] - 0s 70ms/step - loss: 2.5776 - categorical_accuracy: 0.2809
Epoch 5/2000
2/2 [==============================] - 0s 103ms/step - loss: 7.2998 - categorical_accuracy: 0.3017
Epoch 6/2000
2/2 [==============================] - 0s 105ms/step - loss: 12.0845 - categorical_accuracy: 0.2822
Epoch 7/2000
2/2 [==============================] - 0s 138ms/step - loss: 5.3555 - categorical_accuracy: 0.3861
Epoch 8/2000
2/2 [==============================] - 0s 91ms/step - loss: 5.5788 - categorical_accuracy: 0.2224
Epoch 9/2000
2/2 [==============================] - 0s 186ms/step - loss: 14.0392 - categor

# 5. Validating and Predicting the Model

In [10]:
res = model.predict(X_test)
print(actions[np.argmax(res[2])])
print(actions[np.argmax(y_test[2])])


B
B


In [11]:
model.save('action_test.h5')

In [12]:
del model


In [13]:
model.load_weights('action_test.h5')

NameError: name 'model' is not defined